In [1]:
import os
from dotenv import load_dotenv

from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

from chatrag.retriever import create_retrieval_chain, create_retriever_from_csv

In [2]:
load_dotenv()

True

In [4]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="testor_ls"

## Data preparation

In [5]:
retriever_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k", max_tokens=2000)
# retriever_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"vote_average": "float"},
    llm=retriever_llm
)

/Users/itortouch/miniconda3/envs/chatrag_dev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/itortouch/miniconda3/envs/chatrag_dev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  w

In [ ]:
# retriever.get_relevant_documents("I want to watch a movie about space battles.")

# Vectorstore DB

In [6]:
# retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=1000)
retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k", max_tokens=2000)
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

In [ ]:
# print(g_media_sq_retrieval_chain.run("I want to watch a movie about space battles."))

# Tooling

In [7]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence."""

In [8]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    )
]

# Prompt templates

In [9]:
# flake8: noqa
PREFIX = """You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.

Take a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS."""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_kwargs = {
        "system_message": PREFIX,
    }

# Chains

In [10]:
# agent_llm_model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-16k", max_tokens=2000)
agent_llm_model = ChatOpenAI(temperature=0.7, model="gpt-4", max_tokens=2000)

In [11]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True
)

# TESTS

### Run

In [12]:
from langchain.globals import set_debug
set_debug(True)

In [13]:
response = general_chat_agent.run(input="Hi, my name is Aitor")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Hi, my name is Aitor",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Hi, my name is Aitor",
  "chat_history": [],
  "agent_scratchpad": [],
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.\n\nTake a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS.\nHuman: TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Search movies: Movie search tool. The action input must be just t

In [14]:
response = general_chat_agent.run(input="""I would like to watch a movie""")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.\n\nTake a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS.\nHuman: Hi, my name is Aitor\nAI: Hello Aitor! How can I assist you today with movie recommendations?\nHuman: TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Search movies: Movie search tool. The action input must be just topics in a natural language sentence.\n\nRESPONSE FORMAT INSTRUCTIONS\n-----------------

In [15]:
response = general_chat_agent.run(input="""I want to watch a movie about outer space exploration.""")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.\n\nTake a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS.\nHuman: Hi, my name is Aitor\nAI: Hello Aitor! How can I assist you today with movie recommendations?\nHuman: I would like to watch a movie\nAI: Sure, Aitor! Could you please provide more information about the type of movie you want to watch? For instance, are you interested in a specific genre, actor, director, or maybe a specific era?\nHuman: TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be help

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Tue, 16 Jan 2024 14:50:55 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '846722e29e235cfa-MAD', 'alt-svc': 'h3=":443"; ma=86400'}.


[llm/end] [1:chain:AgentExecutor > 4:tool:Search movies > 5:chain:RetrievalQA > 11:chain:StuffDocumentsChain > 12:chain:LLMChain > 13:llm:ChatOpenAI] [400.43s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. Interstellar\n- Justification: Interstellar is a perfect match for the user's request as it revolves around the theme of outer space exploration. With a high vote average of 8.1, it is highly regarded by audiences and critics alike.\n- Score: 8.1\n\n2. 2001: A Space Odyssey\n- Justification: 2001: A Space Odyssey is a classic film that delves into the mysteries of outer space exploration. With a vote average of 7.9, it is highly acclaimed for its visual effects and thought-provoking storyline.\n- Score: 7.9\n\n3. Gravity\n- Justification: Gravity is a thrilling film that showcases the dangers and challenges of outer space exploration. With a vote average of 7.3, it received critical acclaim for its stunning visuals and intense storytelling.\n- Sc

In [16]:
response = general_chat_agent.run(input="I've changed my mind. Would you suggest me a good genre for just relaxing?.")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.\n\nTake a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS.\nHuman: Hi, my name is Aitor\nAI: Hello Aitor! How can I assist you today with movie recommendations?\nHuman: I would like to watch a movie\nAI: Sure, Aitor! Could you please provide more information about the type of movie you want to watch? For instance, are you interested in a specific genre, actor, director, or maybe a specific era?\nHuman: I want to watch a movie about outer space exploration.\nAI: Based on your interest in outer sp

In [17]:
response = general_chat_agent.run(input="Sounds good, give me a movie about it.")
print("\nA partir de aquí la respuesta:\n")
print(response)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are Diverger Assistant, an language model created by OpenAI and used by Diverger, an AI company. You are an assistant expert in movie recommendation.\n\nTake a deep breath, use the TOOLS when needed and ALWAYS follow RESPONSE FORMAT INSTRUCTIONS.\nHuman: Hi, my name is Aitor\nAI: Hello Aitor! How can I assist you today with movie recommendations?\nHuman: I would like to watch a movie\nAI: Sure, Aitor! Could you please provide more information about the type of movie you want to watch? For instance, are you interested in a specific genre, actor, director, or maybe a specific era?\nHuman: I want to watch a movie about outer space exploration.\nAI: Based on your interest in outer sp

In [ ]:
response = general_chat_agent.run(input="By the way, what's your name?.")
print("\nA partir de aquí la respuesta:\n")
print(response)